In [120]:
import os
import sys
project_path = os.path.abspath(os.path.join('../..'))
if project_path not in sys.path:
    sys.path.append(project_path)
# os.path.split(os.path.split(os.getcwd())[0])[0]

In [121]:
import src.CONSTANTS as CNSTS
import os
import src.CONF as CONF
import json
import math
import csv
import talib
import numpy as np
import pandas as pd
import plotly.express as px
from src.stk import zillion
from src.utils import email_info
from datetime import datetime
from dotenv import load_dotenv
from binance.client import Client

In [3]:
# 1. Following constant product formula for market making, x * y = k
# 2. where, x is the amount of asset 1, and y is the amount of asset 2 pooled in.
# 3. Assuming price of asset 1 in relation to asset 2 is m, 
# 4. then y = m * x, therefore k = x * m * x 
# 5. Considering a change in relative price of asset 1 to asset 2 i.e. m -> m',
# 6. Updated state of pooled assets x -> x', y-> y'
# 7. x' * m' * x' = k, therefore x' * m' * x' = x * m * x
# 8. x' = sqr_rt((x * m * x)/m')
# 9. Assuming m' = n * m, where it can be read as new relative price of asset 1 is n% of the current price
# 10. x' = sqr_rt((x * x)/n) 
# 11. y' = k/x'
# 12. Lets consider the $ value of asset 2 unit as v, asset 1 unit $ value (u) would be v * m
# 13. Therefore, total worth of the assets would be x * (v * m) + y * v  
# 14. Considering the updated $ value of asset 2 as u'
# 15. Difference in assets worth would be: (x' * (u' * m') + y' * u') - (x * (u * m) + y * u) 
# 16. This difference is referred to as impermanent loss i.e. comparing the buy and hold strategy vs putting assets in the pool

In [ ]:
# what 1 transaction would bring a change of price of x% in asset1 in the pool
# a buy order of asset1 would increase the price, while a sell order decreases the price
# calculate the amount of asset1 tokens need to be bought from this pool to 
# increase asset1 price by x%
# Assuming a high asset1-asset2 dex pool, i.e. <1% slippage for any invested amount
# roi through liquidity mining is defined by the time spent by the liquidity spent in the pool, which in turn
# gives the amount of fees earned and the governance token rewards
# track binance price continuously, if it changes by 2%, then remove the liquidity
# also note that there could be a lag between the binance prices and the dex price or vice versa (in some cases)
# leading to arbitrage opportunities

In [ ]:
# TODO Classes: DexPool, Asset 

In [4]:
# Experiment with different price curves, 
# e.g. assuming a delta of 1-5% change in reaching the target price change in the assets

In [69]:
def load_env():
    load_dotenv()
    personal_env = os.getenv(CONF.PERSONAL_ENV_FILE)
    load_dotenv(personal_env) 

In [155]:
load_env()

In [156]:
class ConstantProductPool:    
    x = 0
    y = 0
    u = 0
    v = 0
    m = 0
    
    a1_pool_weight = 0.5
    a2_pool_weight = 0.5
    
    def __init__(self, u, v, w):
        self.u = u
        self.x = (w * a1_pool_weight)/self.u
        self.v = v
        self.y = (w * a2_pool_weight)/self.v       
        self.m = u/v
    
    def current_state(self):
        return self.x, self.yy
        
    def product(self, x, y):
        return self.x * self.y

    def worth(self): 
        return self.x * self.u + self.y * self.v

    def update(self, u_dash, v_dash):
        m_dash = u_dash/v_dash
        k = self.product(self.x, self.y)
        self.x = math.sqrt((self.m/m_dash) * self.x * self.x)
        self.y = k/self.x
        self.u = u_dash
        self.v = v_dash
        self.m = m_dash    
        
    def get_user_current_state(self, user_share):
        return user_share * self.x, user_share * self.y
    
    def get_user_worth(self, user_share):
        return user_share * self.worth()

In [157]:
def get_updated_price(price, price_delta):
    return price * (1 + price_delta)

def worth(x, u, y, v): 
    return x * u + y * v

In [160]:
# Example IL calculation
a1_pool_weight = 0.5
a2_pool_weight = 0.5

# Assuming following, user pool in: 10K $, zil at 0.17$, Zil-USD pool
# change of 50% price in Zil

u = 0.17 # $ price of asset 1
v = 1 # $ price of asset 2
delta_u = 0.5 # Percent change
delta_v = 0.0 # Percent change
pool_initial_worth = 1000000000
pool = ConstantProductPool(u, v, pool_initial_worth)
user_initial_worth = 10000
user_share = user_initial_worth/pool_initial_worth
user_x, user_y = pool.get_user_current_state(user_share)
print("Initial user worth", pool.get_user_worth(user_share))
print("user x: ", user_x, " user y: ", user_y)
u_dash = get_updated_price(u, delta_u)
v_dash = get_updated_price(v, delta_v)
pool.update(u_dash, v_dash)
user_x_dash, user_y_dash = pool.get_user_current_state(user_share)
print("user x: ", user_x_dash, " user y: ", user_y_dash)
print("Updated user worth", pool.get_user_worth(user_share))
user_buy_hold_worth = worth(user_x, u_dash, user_y, v_dash)
il = (pool.get_user_worth(user_share) - user_buy_hold_worth)/user_buy_hold_worth
print("Impermanent loss %: ", round(il * 100, 2))
print("User buy and hold worth: ", round(user_buy_hold_worth, 2))
print("Asset 1 % change: ", round(((user_x_dash - user_x)/user_x)*100, 2))
print("Asset 2 % change: ", round(((user_y_dash - user_y)/user_y)*100, 2))

Initial user worth 10000.0
user x:  29411.764705882353  user y:  5000.0
user x:  24014.605321403706  user y:  6123.724356957945
Updated user worth 12247.44871391589
Impermanent loss %:  -2.02
User buy and hold worth:  12500.0
Asset 1 % change:  -18.35
Asset 2 % change:  22.47


In [161]:
def get_binance_client():
    api_key = os.getenv(CONF.BIN_WALLET_1_API_1["API_KEY"])
    api_secret = os.getenv(CONF.BIN_WALLET_1_API_1["API_SECRET"])
    client = Client(api_key, api_secret)
    return client

In [162]:
def get_binance_data(client, start_date, end_date, asset_pair, data_type):    
    # candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_30MINUTE)
    candles = client.get_historical_klines(asset_pair,
                                           Client.KLINE_INTERVAL_15MINUTE,
                                           start_date,
                                           end_date
                                           )
    return candles    

In [180]:
def candles_to_df(candles):
    rows_list = []
    for candle in candles:
        row_dict = {}
        row_dict['Open time'] = candle[0] 
        row_dict['Open'] = candle[1] 
        row_dict['High'] = candle[2] 
        row_dict['Low'] = candle[3] 
        row_dict['Close'] = candle[4] 
        row_dict['Volume'] = candle[5] 
        row_dict['Close time'] = candle[6] 
        row_dict['Quote asset volume'] = candle[7] 
        row_dict['Number of trades'] = candle[8] 
        row_dict['Taker buy base asset volume'] = candle[9] 
        row_dict['Taker buy quote asset volume'] = candle[10] 
        row_dict['Can be ignored'] = candle[11]
     
        rows_list.append(row_dict)
        
    df = pd.DataFrame(rows_list)   
    return df

In [164]:
# def candles_to_df_2(candles):
#     candles_dataframe = pd.DataFrame(columns= ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 
#                                                'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 
#                                                'Taker buy quote asset volume', 'Can be ignored'])
#     for i in range(0, len(candles)):
#         candle_i = candles[i]
#         candles_dataframe.loc[i] = candle_i
#     return candles_dataframe

In [181]:
start_date = "5 Feb, 2021"
end_date = "20 Feb, 2021"
asset_pair = 'ZILUSDT'
data_type = "kline_15m"
client = get_binance_client()
candles = get_binance_data(client, start_date, end_date, asset_pair, data_type)

In [182]:
candles = candles_to_df(candles)

In [183]:
candles.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Can be ignored
0,1612483200000,0.07638000,0.07734000,0.07553000,0.07638000,4534501.60000000,1612484099999,346132.60315300,1660,2236519.20000000,170860.47039900,0
1,1612484100000,0.07635000,0.07780000,0.07627000,0.07726000,1621324.80000000,1612484999999,124862.32032300,951,944165.60000000,72726.83414000,0
2,1612485000000,0.07725000,0.07800000,0.07697000,0.07753000,2741743.10000000,1612485899999,212609.61032600,971,1500829.40000000,116515.48882000,0
3,1612485900000,0.07754000,0.07810000,0.07747000,0.07765000,3084177.50000000,1612486799999,239932.25731900,746,1991830.30000000,155011.05679500,0
4,1612486800000,0.07764000,0.07799000,0.07757000,0.07799000,1896724.00000000,1612487699999,147492.26924300,502,1003836.80000000,78083.14224100,0


In [168]:
candles['Open time stamp'] = candles.apply(lambda x: datetime.fromtimestamp(x["Open time"]/1000.0),axis=1)

In [169]:
candles['Close'] = candles['Close'].astype(float)

In [170]:
# candles['Close'].std()

In [171]:
fig = px.line(candles, x='Open time stamp', y="Close",
              line_shape="spline", render_mode="svg")
fig.show()

In [107]:
total_time = 0
time_spent_in_pool =  0
# To start with, asset 1 is speculative, and asset 2 is a stable one (FIAT). 
# Idea is to maximize time spent by the liquidity in the pool, while minimizing the 
# change in the percentage of the asset 1. 
# With any of the implemented approaches, aim is to beat the losses (asset 1 loss and/or IL)
# occuring with doing no dynamic liquidity additions/removals.
# Approach 1: Put a constraint on the change of asset 1 %, and remove liquidity if the change 
# is more than the threshold. 

In [173]:
u = candles.iloc[0]['Close']
v = 1
pool_initial_worth = 100000000
pool = ConstantProductPool(u, v, pool_initial_worth)
user_initial_worth = 10000
user_share = user_initial_worth/pool_initial_worth
ils = []

u_losses = []
u_start_delta = []
u_recent = u
user_x, user_y = pool.get_user_current_state(user_share)
for index, row in candles.iterrows():
    u_dash = row['Close']
    v_dash = v
    pool.update(u_dash, v_dash)
    user_x_dash, user_y_dash = pool.get_user_current_state(user_share)    
    user_buy_hold_worth = worth(user_x, u_dash, user_y, v_dash)
    il = (pool.get_user_worth(user_share) - user_buy_hold_worth)/user_buy_hold_worth
    ils.append(round(il * 100, 2))
    u_losses.append(round(((user_x_dash - user_x)/user_x)*100, 2))

In [175]:
ils[len(ils)-1]

-4.81

In [176]:
u_losses[len(u_losses)-1]

-27.13

In [177]:
result = pd.DataFrame({"IL": ils, "Asset1_Loss": u_losses})

In [178]:
result

,IL,Asset1_Loss
0,0.00,0.00
1,-0.00,-0.57
2,-0.00,-0.74
3,-0.00,-0.82
4,-0.01,-1.04
...,...,...
1431,-4.15,-25.41
1432,-4.37,-26.00
1433,-4.98,-27.55
1434,-5.36,-28.46
